# Working with more than one graph
Each graph has a unique identifier that can be used in some APIs. The optional `graph_id` parameter specifies a particular graph to be used, rather than just the currently active graph.

In [ ]:
import random
import pandas as pd
from IPython.display import display, Image
import constellation_client

cc = constellation_client.Constellation()

In [ ]:
N = 3

In [ ]:
# Create some new graphs and get their graph ids.
#
for _ in range(N):
    cc.new_graph()

graphs = cc.list_graphs()
display(graphs)

graph_ids = [g['id'] for g in graphs]
display(graph_ids)

In [ ]:
# Populate the graphs.
#
for i, gid in enumerate(graph_ids):
    df = pd.DataFrame({
        'source.Label': [f'user_a{i}@example.com<Email>', f'user_b{i}@example.com<Email>', f'user_c{i}@example.com<Email>'],
        'destination.Label': [None, f'user_c{i}@example.com<Email>', f'user_b{i+1}@example.com<Email>'],
        'transaction.directed':[True, True, True]
    })
    print(f'Put dataframe {i} to graph {gid} ...')
    cc.put_dataframe(df, graph_id=gid)

In [ ]:
# Get the data from each graph and merge them into a new graph.
#
dfs = []
for gid in graph_ids:
    print(f'Get dataframe from graph {gid} ...')
    df = cc.get_dataframe(graph_id=gid, attrs=['source.Label', 'destination.Label', 'transaction.directed'])
    dfs.append(df)

df = pd.concat(dfs)
display(df)

gid_new = cc.new_graph()
cc.put_dataframe(df, graph_id=gid_new)

In [ ]:
def rand_color():
    return '#' + ''.join(f'{random.randrange(0, 256):02x}' for _ in range(3))

In [ ]:
# Run some plugins on each graph.
#
for gid,df in zip(graph_ids, dfs):
    vx_ids = list(df['source.[id]'])
    args = {
        'BlazeUtilities.vertex_ids': vx_ids,
        'BlazeUtilities.color': rand_color()
    }
    print(f'Graph {gid} ...')
    cc.run_plugin('addcustomblaze', graph_id=gid, args=args)
    cc.run_plugin('arrangeincircle', graph_id=gid)
    cc.run_plugin('resetview', graph_id=gid)

In [ ]:
# Set some graph attributes.
#
for gid in graph_ids:
    labels = f'Type;{rand_color()};0.5|Label;{rand_color()};1'
    print(f'Graph {gid} {labels}')
    df = pd.DataFrame({'node_labels_top': [labels], 'background_color':[rand_color()]})
    cc.set_graph_attributes(df, graph_id=gid)

In [ ]:
# Get the graph attributes.
#
for gid in graph_ids:
    df = cc.get_graph_attributes(graph_id=gid)
    labels = df['node_labels_top'][0]
    print(f'Graph {gid} {labels}')

A graph image must be taken from the active graph, so `get_graph_image()` doesn't have a graph_id parameter.

In [ ]:
for gid in graph_ids:
    cc.set_current_graph(gid)
    buf = cc.get_graph_image()
    display(f'Graph {gid}')
    display(Image(buf))